* Import librairies

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms, models # import a library that contains pretrained models
from torchsummary import summary

* Download Multi-class Weather Dataset from kaggle

In [2]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d pratik2901/multiclass-weather-dataset
!unzip 'multiclass-weather-dataset.zip'
!kaggle datasets download -d pratik2901/multiclass-weather-dataset
%cd "/content/Multi-class Weather Dataset"

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/pratik2901/multiclass-weather-dataset
License(s): Attribution 4.0 International (CC BY 4.0)
100% 91.0M/91.4M [00:05<00:00, 24.1MB/s]
100% 91.4M/91.4M [00:05<00:00, 18.8MB/s]
Archive:  multiclass-weather-dataset.zip
  inflating: Multi-class Weather Dataset/Cloudy/cloudy1.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy10.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy100.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy101.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy102.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy103.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy104.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy105.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy106.jpg  
  inflating: Multi-class Weather D

* Load all images from dataset


In [3]:
dataset = torchvision.datasets.ImageFolder(
        root = "/content/Multi-class Weather Dataset", # data_path is path to directory
        transform = None,
)

print(dataset)

Dataset ImageFolder
    Number of datapoints: 1125
    Root location: /content/Multi-class Weather Dataset


* Divide dataset into train (70%) and test (30%) subsets

In [4]:
train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

print(len(train_dataset), "train samples and", len(test_dataset), "test samples")

787 train samples and 338 test samples


* Define transformations on train and tests subsets

In [5]:
IMAGE_SIZE = 128

train_transform = transforms.Compose(
    [
        transforms.Resize((IMAGE_SIZE,IMAGE_SIZE)),
        transforms.RandomRotation(10),
        transforms.RandomResizedCrop(64),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]
)

test_transform = transforms.Compose(
    [
        transforms.Resize((IMAGE_SIZE,IMAGE_SIZE)),
        transforms.ToTensor(),
    ]
)

train_dataset.dataset.transform = train_transform
test_dataset.dataset.transform = test_transform


* Create DataLoader object for train and test subsets

In [6]:
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = 32,
                                           shuffle = True
)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = 32,
                                          shuffle = False
)

for batch_num, (images, targets) in enumerate(train_loader):
  print("Batch image size", images.shape, "---- Batch label size", targets.shape)

Batch image size torch.Size([32, 3, 128, 128]) ---- Batch label size torch.Size([32])
Batch image size torch.Size([32, 3, 128, 128]) ---- Batch label size torch.Size([32])
Batch image size torch.Size([32, 3, 128, 128]) ---- Batch label size torch.Size([32])
Batch image size torch.Size([32, 3, 128, 128]) ---- Batch label size torch.Size([32])
Batch image size torch.Size([32, 3, 128, 128]) ---- Batch label size torch.Size([32])
Batch image size torch.Size([32, 3, 128, 128]) ---- Batch label size torch.Size([32])
Batch image size torch.Size([32, 3, 128, 128]) ---- Batch label size torch.Size([32])
Batch image size torch.Size([32, 3, 128, 128]) ---- Batch label size torch.Size([32])
Batch image size torch.Size([32, 3, 128, 128]) ---- Batch label size torch.Size([32])
Batch image size torch.Size([32, 3, 128, 128]) ---- Batch label size torch.Size([32])
Batch image size torch.Size([32, 3, 128, 128]) ---- Batch label size torch.Size([32])
Batch image size torch.Size([32, 3, 128, 128]) ---- Ba

* Load the pretrained VGG16 model and turn it to CUDA device if available

In [37]:
vgg = models.vgg16(pretrained= True) # import vgg16 model

if torch.cuda.is_available():
  device = "cuda"
else:
  device = "cpu"

vgg = vgg.to(device)

print(vgg)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

### Adapt VGG16 to our downstream task (modify classification part)

* Set the input size of vgg as (3, 128, 128)
* Set the configuration of the classifier part as follow:
  - hidden layer of 512 neurons with ReLU
  - hidden layer of 4 neurons with SoftMax

In [39]:
# set the vgg input size as (3, 128, 128)
vgg.input_size = (3, 128, 128)

# change the classifier part of vgg
vgg.classifier = nn.Sequential(
    nn.Linear(in_features=25088, out_features= 512 ),
    nn.ReLU(),
    nn.Linear(in_features= 512 , out_features=4),
    nn.Softmax(dim=1),
)

print(vgg)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

* Set the features layer weights no-trainable

In [40]:
for param in vgg.features.parameters():
  param.requires_grad = False

* Define hyper-parameters (Loss function and optimizer)

In [41]:
# Loss function
loss_function = nn.CrossEntropyLoss()

# optimizer Adam
optimizer = torch.optim.Adam(params = vgg.parameters(), lr=1e-4)

* Run the training Loop

In [42]:
# !export CUDA_LAUNCH_BLOCKING=1

epochs = 20

# turn vgg model to CUDA device (after being finetuned)
vgg.to(device)

# set the model to train mode
vgg.train()


for epoch in range(epochs):

    # define a list to store the losses of the different batches
    losses = []

    # iter over patches
    for batch_num, (train_images, train_targets) in enumerate(train_loader):

        # initialize the optimizer (Zero the gradients)
        optimizer.zero_grad()

        # extract images and targets from input_data (Get inputs)
        train_images = train_images.to(device)
        train_targets = train_targets.to(device)

        # Propagate the images and return the predicted output (Perform forward pass)
        outputs = vgg(train_images)

        # calculate the loss between actual and predicted outputs (Compute loss)
        loss = loss_function(outputs, train_targets)
        losses.append(loss.item())

        # Prpagate the gradient of the loss in reverse (Perform backward pass)
        loss.backward()

        # update the model parameters (Perform optimization)
        optimizer.step()

    print('Epoch %d -- Loss %6.4f' % (epoch, sum(losses)/len(losses)))

# Process is complete.
print('Training process has finished.')

Epoch 0 -- Loss 0.9756
Epoch 1 -- Loss 0.8013
Epoch 2 -- Loss 0.7715
Epoch 3 -- Loss 0.7615
Epoch 4 -- Loss 0.7569
Epoch 5 -- Loss 0.7540
Epoch 6 -- Loss 0.7510
Epoch 7 -- Loss 0.7506
Epoch 8 -- Loss 0.7502
Epoch 9 -- Loss 0.7500
Epoch 10 -- Loss 0.7497
Epoch 11 -- Loss 0.7504
Epoch 12 -- Loss 0.7494
Epoch 13 -- Loss 0.7493
Epoch 14 -- Loss 0.7492
Epoch 15 -- Loss 0.7500
Epoch 16 -- Loss 0.7490
Epoch 17 -- Loss 0.7489
Epoch 18 -- Loss 0.7482
Epoch 19 -- Loss 0.7480
Training process has finished.


* Save the trained model

In [43]:
torch.save(vgg, 'vgg.pth')

* Load the model and run evaluation

In [46]:
# load model saved in "vgg16_model.pth"
vgg16_model =torch.load('vgg.pth')

# turn the pretrained model to cuda device
if torch.cuda.is_available():
  device = "cuda"
else:
  device = "cpu"

vgg16_model = vgg16_model.to(device)

# initialize evaluation metrics
total_samples = len(test_loader.dataset)
correct_samples = 0
total_loss = 0

# set the model in evaluation mode
vgg16_model.eval()

# iter over batches
for num_batch, (test_images, test_targets) in enumerate(test_loader):

  # extract images and targets from test_data (Get inputs)
  test_images = test_images.to(device)
  test_targets = test_targets.to(device)

  # calculate logits
  outputs = vgg16_model(test_images)

  # calculate loss
  loss = loss_function(outputs, test_targets)
  total_loss += loss.item()

  # predict classes from outputs
  pred_targets = outputs.argmax(dim=1)

  # calculate correct predictions
  nb_correct = pred_targets.eq(test_targets).sum()
  correct_samples += nb_correct

avg_loss = total_loss / total_samples
val_accuracy = (correct_samples / total_samples)

print('\nAverage test loss: ' + '{:.4f}'.format(avg_loss) +
          '  val_accuracy:' + '{:.4f}'.format(val_accuracy) + '\n')

<ipython-input-46-da5cb34ed090>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vgg16_model =torch.load('vgg.pth')



Average test loss: 0.0255  val_accuracy:0.9645

